# Example LePHARE run


In this example notebook we demonstrate a run on the CLAUDS data (Desprez et al. 2023) data. This notebook is set in order to show the most basic LePHARE functionalities.

This notebook should demonstrate the procedure for 
* reading/updating the configuration parameters
* creating an input table in the appropriate format
* build the  template library
* run the photo-z
* inspect the results, including the best fit SEDs 

## Load the necessarily packages, including LePHARE

In [ ]:
# lephare must be installed if not already
#!pip install lephare 

In [ ]:
import lephare as lp
from astropy.table import Table
import numpy as np
from matplotlib import pylab as plt

## Load the CLAUDS data

We use photometry from HSC-CLAUDS presented in [Sawicki et al. 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.489.5202S/abstract). We limit our example to the use of the ugrizy bands. This includes a sample of AGN that are detected in the X-ray by [Marchesi et al. 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...817...34M/abstract). Spectroscopic redhsifts are from [Khostovan et al. 2025](https://arxiv.org/abs/2503.00120). These also provide a broad line AGN sample which can be compared and contrasted to X-ray selected AGN.


Our example use an astropy table as input. This can be done using the standard column order:
id, flux0, err0, flux1, err1,..., context, zspec, arbitrary_string. A simple example table with two filters might look like this:
|  id | flux_filt1  |  fluxerr_filt1 |  flux_filt2  |  fluxerr_filt2 | context | zspec | string_data |
|---|---|---|---|---|---|---|---|
|  0 | 1.e-31  | 1.e-32  | 1.e-31  | 2.e-32  | 3 | NaN | "This is just a note" |
|  1 | 2.e-31  |  1.e-32 | 1.e-31  | 2.e-32  |3 | 1. | "This has a specz" |
|  2 | 2.e-31 | 1.e-32  | 2.e-31  | 2.e-32  | 2 | NaN| "This context only uses the second filter" |

The context detemermines which bands are used but can be -99 or a numpy.nan. We do not need to have units on the flux columns but LePHARE assumes they are in erg /s /cm**2 / Hz if we are using fluxes. The number of columns must be two times the number of filters plus the four additional columns.

This input table **must use** the standard column ordering to determine column meaning. This odering depends on the filter order in the config FILTER_LIST value. 

In [ ]:
input_lp=Table.read('input_flux_clauds_gal_021125.fits')

In [ ]:
input_lp[:5]

Show the different populations of sources present in the catalogue (normal galaxies, x-ray AGN, broad line AGN)

In [ ]:
for s in np.unique(input_lp['string_input']):
    print(s, np.sum(input_lp['string_input']==s))

Trim the catalogue to keep only galaxies for the moment

In [ ]:
is_galaxy = input_lp['string_input'] == 'galaxy'
galaxy_indices = np.where(is_galaxy)[0]
input_lp_gal = input_lp[galaxy_indices]

## Update the config

We set up a specific CLAUDS configuration file as a basis. We can update the various keywords in the notebook if desired. 

In [ ]:
config = lp.read_config("CLAUDS.para")

# Example of updating one parameter, which is the grid in redshift
config.update({
    'Z_STEP': '0.02,0.,6.',
})

## Download the data required for the run


In [ ]:
# If you want all the files with many different filters and templates etc
# lp.data_retrieval.get_auxiliary_data(clone=False)

This function checks that all the necessary data are in LEPHAREDIR. 
It will use the automated download functionality if some files are missing.

In [ ]:
# We need to download the required files
lp.data_retrieval.get_auxiliary_data(keymap=config)

Plot the filters as a sanity check

In [ ]:
for n, f in enumerate(config['FILTER_LIST'].value.split(',')):
    data = Table.read(f"{lp.LEPHAREDIR}/filt/{f}", format="ascii")
    plt.plot(data[data.colnames[0]], data[data.colnames[1]], label=f)
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.xlabel("Wavelength [Angstrom]")
plt.ylabel("Transmission")

## Create the full library of modeled fluxes

These are the key preparatory stages to build the library of modeled fluxes. 
It calculates the filters in the LePHARE format, calculate the library of SEDs and finally calculate the library of magnitudes for all the models. The *prepare* method runs *filter*, *sedtolib*, and *mag_gal* that would be run independently at the command line. These are all explained in detail in the [documentation](https://lephare.readthedocs.io/en/latest/original.html#detailed-lephare-user-manual).

In [ ]:
lp.prepare(config,qso_config=None)

## Run the photo-z

Only galaxy and star templates for the mome


In [ ]:
config.update({
    'ZPHOTLIB': 'GAL_CLAUDS,STAR_CLAUDS',
})

In [ ]:
config.update({
    'AUTO_ADAPT': 'YES',
    'NZ_PRIOR': '4',
})

Finally we run the main fitting process which is equivalent to *zphota* when using the command line. 


In [ ]:
output, photozlist=lp.process(config, input_lp_gal,write_outputs=True)

## Take a quick look at the output

You can see the main columns present int he output. Some columns regarding phsyical parameters are not present because we have not computed them in a standard run. you can see an example [here](https://lephare.readthedocs.io/en/latest/notebooks/Typical_use_case_physicalParameters.html) for computing them.

In [ ]:
output[:5]

## Investigate the results

Here are some quick checks of the results

In [ ]:
z_max=4
plt.axis([0,z_max,0,z_max])

gal = (output['ZSPEC']>0.002) & (output['ZSPEC']<z_max) & (output['Z_BEST']>0.002) & (output['Z_BEST']<z_max)
zphot = output['Z_BEST'][gal]
zspec = output['ZSPEC'][gal]
plt.scatter(zphot,zspec,s=0.2)

#Trace the limits 0.15(1+z)
x_zs = np.array([0,z_max])
plt.plot(x_zs, x_zs*1.15+0.15, 'c--')
plt.plot(x_zs, x_zs, 'r-')
plt.plot(x_zs, x_zs*0.85-0.15, 'c--')

# Statistics
delz_norm=(zphot-zspec)/(1+zspec)
delz_norm2= delz_norm-np.median(delz_norm)
nmad = 1.48*np.median(np.abs(delz_norm2))
cond_outl = ( abs(delz_norm) > 0.15 )
outl_rate = len(delz_norm[cond_outl]) / float(len(delz_norm))
plt.annotate(r'$\eta  ='+str(100*round(outl_rate,3))+'  \%$\n'+'$ \sigma_{\Delta z /(1+z)}  = '+str(round(nmad,5))+'$',xy=(0.1*z_max,0.8*z_max),color="black", fontsize=15)

plt.xlabel("$z_{spec}$")
plt.ylabel("$z_{phot}$")

In [ ]:
plt.hist(output['ZSPEC'][gal],bins=20,density=True,label='Galaxies $z_{spec}$',alpha=0.5)
plt.hist(output['Z_BEST'][gal],bins=20,density=True,label='Galaxies $z_{phot}$',alpha=0.5)
plt.legend()
plt.xlabel('$z$')

In [ ]:
#gal
from os import listdir
from os.path import isfile, join

listname = [f for f in listdir("save_spec") if isfile(join("save_spec", f))]
# Lets just look at the top 10
for namefile in listname[:10]:
    lp.plotspec("save_spec/" + str(namefile))